# కుటుంబ సంబంధాల ఆంటాలజీ

ఈ ఉదాహరణ [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) లో భాగంగా ఉంది, మరియు ఇది [ఈ బ్లాగ్ పోస్ట్](https://habr.com/post/270857/) నుండి ప్రేరణ పొందింది.

కుటుంబంలోని వ్యక్తుల మధ్య వివిధ సంబంధాలను గుర్తుంచుకోవడం నాకు ఎప్పుడూ కష్టం అవుతుంది. ఈ ఉదాహరణలో, కుటుంబ సంబంధాలను నిర్వచించే ఒక ఆంటాలజీని, మరియు వాస్తవ వంశావళి చెట్టును తీసుకుని, ఆ తర్వాత అన్ని బంధువులను కనుగొనడానికి ఆటోమేటిక్ ఇన్ఫరెన్స్ ఎలా చేయవచ్చో చూపిస్తాము.

### వంశావళి చెట్టు పొందడం

ఉదాహరణగా, మనం [రోమానోవ్ సార కుటుంబం](https://en.wikipedia.org/wiki/House_of_Romanov) యొక్క వంశావళి చెట్టును తీసుకుంటాము. కుటుంబ సంబంధాలను వివరించడానికి అత్యంత సాధారణ ఫార్మాట్ [GEDCOM](https://en.wikipedia.org/wiki/GEDCOM). మనం రోమానోవ్ కుటుంబ వంశావళిని GEDCOM ఫార్మాట్‌లో తీసుకుంటాము:


In [1]:
!head -15 data/tsars.ged

0 HEAD
1 CHAR UTF8
1 GEDC
2 VERS 5.5
0 @0@ INDI
1 NAME Mihail Fedorovich /Romanov/
1 SEX M
1 BIRT
2 DATE 1613
1 DEAT 
2 DATE 1645
1 FAMS @41@
0 @1@ INDI
1 NAME Evdokija Lukjanovna /Streshneva/
1 SEX F


GEDCOM ఫైల్ ఉపయోగించడానికి, మనం `python-gedcom` లైబ్రరీని ఉపయోగించవచ్చు:


In [2]:
import sys
!{sys.executable} -m pip install python-gedcom

ఈ లైబ్రరీ ఫైల్ పార్సింగ్‌తో సంబంధిత కొన్ని సాంకేతిక సమస్యలను తొలగిస్తుంది, కానీ ఇది ఇంకా మనకు చెట్టులోని అన్ని వ్యక్తులు మరియు కుటుంబాలకు తక్కువ స్థాయి యాక్సెస్ ఇస్తుంది. ఫైల్‌ను ఎలా పార్స్ చేయాలో, మరియు అన్ని వ్యక్తుల జాబితాను ఎలా చూపించాలో ఇక్కడ ఉంది:


In [3]:
from gedcom.parser import Parser
from gedcom.element.individual import IndividualElement
from gedcom.element.family import FamilyElement
g = Parser()
g.parse_file('data/tsars.ged')

In [4]:
d = g.get_element_dictionary()
[ (k,v.get_name()) for k,v in d.items() if isinstance(v,IndividualElement)]

[('@0@', ('Mihail Fedorovich', 'Romanov')),
 ('@1@', ('Evdokija Lukjanovna', 'Streshneva')),
 ('@2@', ('Aleksej Mihajlovich', 'Romanov')),
 ('@3@', ('Marija Ilinichna', 'Miloslavskaja')),
 ('@4@', ('Natalja Kirillovna', 'Naryshkina')),
 ('@5@', ('Marfa Matveevna', 'Apraksina')),
 ('@6@', ('Fedor Alekseevich', 'Romanov')),
 ('@7@', ('Sofja Aleksevna', 'Romanova')),
 ('@8@', ('Ivan V Alekseevich', 'Romanov')),
 ('@9@', ('Praskovja Fedorovna', 'Saltykova')),
 ('@10@', ('Ekaterina Ivanovna', 'Romanova')),
 ('@11@', ('Anna Ivanovna', 'Romanova')),
 ('@12@', ('Fridrih Vilgelm', 'Kurlandskij')),
 ('@13@', ('Karl Leopold', 'Meklenburg-Shverinskij')),
 ('@14@', ('Anna Leopoldovna', 'Meklenburg-Shverinskaja')),
 ('@15@', ('Anton Ulrih', 'Braunshvejg-Volfenbjuttelskij')),
 ('@16@', ('Ivan VI Antonovich', 'Braunshvejg-Volfenbjuttelskij')),
 ('@17@', ('Petr I Alekseevich', 'Romanov')),
 ('@18@', ('Evdokija Fedorovna', 'Lopuhina')),
 ('@19@', ('Ekaterina I Alekseevna', 'Mihajlova')),
 ('@20@', ('Ale

ఇక్కడ కుటుంబాల గురించి సమాచారం ఎలా పొందాలో ఉంది. గమనించండి ఇది మనకు **గుర్తింపులు** జాబితాను ఇస్తుంది, మరియు మనకు మరింత స్పష్టత కావాలంటే వాటిని పేర్లుగా మార్చుకోవాలి:


In [5]:
d = g.get_element_dictionary()
[ (k,[x.get_value() for x in v.get_child_elements()]) for k,v in d.items() if isinstance(v,FamilyElement)]

[('@41@', ['@0@', '@1@', '@2@']),
 ('@42@', ['@2@', '@3@', '@6@', '@7@', '@8@']),
 ('@43@', ['@8@', '@9@', '@10@', '@11@']),
 ('@44@', ['@13@', '@10@', '@14@']),
 ('@45@', ['@15@', '@14@', '@16@']),
 ('@46@', ['@2@', '@4@', '@17@']),
 ('@47@', ['@17@', '@18@', '@20@']),
 ('@48@', ['@20@', '@21@', '@22@']),
 ('@49@', ['@17@', '@19@', '@23@', '@24@']),
 ('@50@', ['@25@', '@23@', '@26@']),
 ('@51@', ['@26@', '@27@', '@28@']),
 ('@52@', ['@28@', '@30@', '@31@', '@33@']),
 ('@53@', ['@33@', '@34@', '@35@']),
 ('@54@', ['@35@', '@36@', '@37@']),
 ('@55@', ['@37@', '@38@', '@39@'])]

### కుటుంబ ఆంటాలజీ పొందడం

తర్వాత, [కుటుంబ ఆంటాలజీ](https://raw.githubusercontent.com/blokhin/genealogical-trees/master/data/header.ttl) ను సెమాంటిక్ వెబ్ ట్రిప్లెట్ల సెట్‌గా నిర్వచించిన దాన్ని చూద్దాం. ఈ ఆంటాలజీ `isUncleOf`, `isCousinOf` వంటి సంబంధాలను నిర్వచిస్తుంది. ఆ సంబంధాలన్నీ ప్రాథమిక ప్రెడికేట్లు అయిన `isMotherOf`, `isFatherOf`, `isBrotherOf` మరియు `isSisterOf` ఆధారంగా నిర్వచించబడ్డాయి. ఆంటాలజీ ఉపయోగించి మిగతా అన్ని సంబంధాలను ఆటోమేటిక్ రీజనింగ్ ద్వారా తేల్చుకుంటాము.

ఇక్కడ `isAuntOf` ప్రాపర్టీ యొక్క ఉదాహరణ నిర్వచనం ఉంది, ఇది `isSisterOf` మరియు `isParentOf` యొక్క సంయోజనంగా నిర్వచించబడింది (*అంటీ అనేది ఒకరి తల్లిదండ్రుల సోదరి*).


In [6]:
!head -20 data/onto.ttl

@prefix fhkb: <http://www.example.com/genealogy.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.example.com/genealogy.owl#> a owl:Ontology .

fhkb:DomainEntity a owl:Class .

fhkb:Man a owl:Class ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( fhkb:Person [ a owl:Restriction ;
                        owl:onProperty fhkb:hasSex ;
                        owl:someValuesFrom fhkb:Male ] ) ] .

fhkb:Woman a owl:Class ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( fhkb:Person [ a owl:Restriction ;


### తర్కానికి ఒంటాలజీ నిర్మాణం

సరళత కోసం, మేము ఒక ఒంటాలజీ ఫైల్‌ను సృష్టిస్తాము, ఇందులో ఫ్యామిలీ ఒంటాలజీ నుండి అసలు నియమాలు మరియు మా GEDCOM ఫైల్ నుండి వ్యక్తుల గురించి వాస్తవాలు ఉంటాయి. మేము GEDCOM ఫైల్‌ను పరిశీలించి కుటుంబాలు మరియు వ్యక్తుల గురించి సమాచారం సేకరించి, వాటిని త్రిపుటాలుగా మార్చుతాము.


In [7]:
!cp data/onto.ttl .

gedcom_dict = g.get_element_dictionary()
individuals, marriages = {}, {}

def term2id(el):
    return "i" + el.get_pointer().replace('@', '').lower()

out = open("onto.ttl","a")

for k, v in gedcom_dict.items():
    if isinstance(v,IndividualElement):
        children, siblings = set(), set()
        idx = term2id(v)

        title = v.get_name()[0] + " " + v.get_name()[1]
        title = title.replace('"', '').replace('[', '').replace(']', '').replace('(', '').replace(')', '').strip()

        own_families = g.get_families(v, 'FAMS')
        for fam in own_families:
            children |= set(term2id(i) for i in g.get_family_members(fam, "CHIL"))

        parent_families = g.get_families(v, 'FAMC')
        if len(parent_families):
            for member in g.get_family_members(parent_families[0], "CHIL"): # NB adoptive families i.e len(parent_families)>1 are not considered (TODO?)
                if member.get_pointer() == v.get_pointer():
                    continue
                siblings.add(term2id(member))

        if idx in individuals:
            children |= individuals[idx].get('children', set())
            siblings |= individuals[idx].get('siblings', set())
        individuals[idx] = {'sex': v.get_gender().lower(), 'children': children, 'siblings': siblings, 'title': title}

    elif isinstance(v,FamilyElement):
        wife, husb, children = None, None, set()
        children = set(term2id(i) for i in g.get_family_members(v, "CHIL"))

        try:
            wife = g.get_family_members(v, "WIFE")[0]
            wife = term2id(wife)
            if wife in individuals: individuals[wife]['children'] |= children
            else: individuals[wife] = {'children': children}
        except IndexError: pass
        try:
            husb = g.get_family_members(v, "HUSB")[0]
            husb = term2id(husb)
            if husb in individuals: individuals[husb]['children'] |= children
            else: individuals[husb] = {'children': children}
        except IndexError: pass

        if wife and husb: marriages[wife + husb] = (term2id(v), wife, husb)

for idx, val in individuals.items():
    added_terms = ''
    if val['sex'] == 'f':
        parent_predicate, sibl_predicate = "isMotherOf", "isSisterOf"
    else:
        parent_predicate, sibl_predicate = "isFatherOf", "isBrotherOf"
    if len(val['children']):
        added_terms += " ;\n    fhkb:" + parent_predicate + " " + ", ".join(["fhkb:" + i for i in val['children']])
    if len(val['siblings']):
        added_terms += " ;\n    fhkb:" + sibl_predicate + " " + ", ".join(["fhkb:" + i for i in val['siblings']])
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing%s ;\n    rdfs:label \"%s\" .\n" % (idx, added_terms, val['title']))

for k, v in marriages.items():
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing ;\n    fhkb:hasFemalePartner fhkb:%s ;\n    fhkb:hasMalePartner fhkb:%s .\n" % v)

out.write("[] a owl:AllDifferent ;\n    owl:distinctMembers (")
for idx in individuals.keys():
    out.write("    fhkb:" + idx)
for k, v in marriages.items():
    out.write("    fhkb:" + v[0])
out.write("    ) .")
out.close()

In [8]:
!tail onto.ttl

    fhkb:hasFemalePartner fhkb:i34 ;
    fhkb:hasMalePartner fhkb:i33 .
fhkb:i54 a owl:NamedIndividual, owl:Thing ;
    fhkb:hasFemalePartner fhkb:i36 ;
    fhkb:hasMalePartner fhkb:i35 .
fhkb:i55 a owl:NamedIndividual, owl:Thing ;
    fhkb:hasFemalePartner fhkb:i38 ;
    fhkb:hasMalePartner fhkb:i37 .
[] a owl:AllDifferent ;
    owl:distinctMembers (    fhkb:i0    fhkb:i1    fhkb:i2    fhkb:i3    fhkb:i4    fhkb:i5    fhkb:i6    fhkb:i7    fhkb:i8    fhkb:i9    fhkb:i10    fhkb:i11    fhkb:i12    fhkb:i13    fhkb:i14    fhkb:i15    fhkb:i16    fhkb:i17    fhkb:i18    fhkb:i19    fhkb:i20    fhkb:i21    fhkb:i22    fhkb:i23    fhkb:i24    fhkb:i25    fhkb:i26    fhkb:i27    fhkb:i28    fhkb:i29    fhkb:i30    fhkb:i31    fhkb:i32    fhkb:i33    fhkb:i34    fhkb:i35    fhkb:i36    fhkb:i37    fhkb:i38    fhkb:i39    fhkb:i40    fhkb:i41    fhkb:i42    fhkb:i43    fhkb:i44    fhkb:i45    fhkb:i46    fhkb:i47    fhkb:i48    fhkb:i49    fhkb:i50    fhkb:i51    fhkb:i52    fhkb:i53    fhkb:

### అనుమానాన్ని చేయడం

ఇప్పుడు మనం ఈ ఆంటాలజీని అనుమానానికి మరియు ప్రశ్నించడానికి ఉపయోగించగలగాలి. మనం RDF గ్రాఫ్‌ను వివిధ ఫార్మాట్లలో చదవడం, దానిని ప్రశ్నించడం మొదలైన వాటికి [RDFLib](https://github.com/RDFLib) లైబ్రరీని ఉపయోగిస్తాము.

తార్కిక అనుమానానికి, మనం [OWL-RL](https://github.com/RDFLib/OWL-RL) లైబ్రరీని ఉపయోగిస్తాము, ఇది RDF గ్రాఫ్ యొక్క **క్లోజర్**ను నిర్మించడానికి అనుమతిస్తుంది, అంటే అనుమానించగల అన్ని సాధ్యమైన భావనలు మరియు సంబంధాలను జోడిస్తుంది.


In [10]:
!{sys.executable} -m pip install rdflib
!{sys.executable} -m pip install git+https://github.com/RDFLib/OWL-RL.git

  Cloning https://github.com/RDFLib/OWL-RL.git to /tmp/pip-req-build-lbfzwi3m
  Running command git clone --filter=blob:none --quiet https://github.com/RDFLib/OWL-RL.git /tmp/pip-req-build-lbfzwi3m
  Resolved https://github.com/RDFLib/OWL-RL.git to commit a77e1791b88b54aace609bc6000aac14c7add4ff
  Preparing metadata (setup.py) ... done


ఒంటాలజీ ఫైల్‌ను తెరవండి మరియు అది ఎంతమంది ట్రిప్లెట్లు కలిగి ఉందో చూద్దాం:


In [11]:
import rdflib
from owlrl import DeductiveClosure, OWLRL_Extension

g = rdflib.Graph()
g.parse("onto.ttl", format="turtle")

print("Triplets found:%d" % len(g))

Triplets found:669


ఇప్పుడు క్లోజర్‌ను నిర్మిద్దాం, మరియు ట్రిప్లెట్ల సంఖ్య ఎలా పెరుగుతుందో చూద్దాం:


In [12]:
DeductiveClosure(OWLRL_Extension).expand(g)
print("Triplets after inference:%d" % len(g))

Triplets after inference:4246


### బంధువులను విచారణ చేయడం

ఇప్పుడు మనం గ్రాఫ్‌ను విచారించి వ్యక్తుల మధ్య వివిధ సంబంధాలను చూడవచ్చు. మనం **SPARQL** భాషను `query` పద్ధతితో కలిసి ఉపయోగించవచ్చు. మన సందర్భంలో, మన కుటుంబ వృక్షంలో ఉన్న అన్ని **మామల్ని** చూద్దాం:


In [13]:
qres = g.query(
    """SELECT DISTINCT ?aname ?bname
       WHERE {
          ?a fhkb:isUncleOf ?b .
          ?a rdfs:label ?aname .
          ?b rdfs:label ?bname .
       }""")

for row in qres:
    print("%s is uncle of %s" % row)

Fedor Alekseevich Romanov is uncle of Ekaterina Ivanovna Romanova
Aleksandr I Pavlovich Romanov is uncle of Aleksandr II Nikolaevich Romanov
Fedor Alekseevich Romanov is uncle of Anna Ivanovna Romanova


వివిధ ఇతర కుటుంబ సంబంధాలతో ప్రయోగాలు చేయడానికి స్వేచ్ఛగా ఉండండి. ఉదాహరణకు, మీరు `isAncestorOf` సంబంధాన్ని చూడవచ్చు, ఇది నిరంతరం ఒక వ్యక్తి యొక్క అన్ని పూర్వీకులను నిర్వచిస్తుంది.

చివరగా, మనం శుభ్రపరచుకుందాం!


In [14]:
!rm onto.ttl

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలో అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారితీసే అర్థాలు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
